In [1]:
pip install numpy==1.22.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 37.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
chex 0.1.84 requires numpy>=1.24.1, but you have numpy 1.22.4 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import statsmodels.api as sm
import tensorflow as tf
from tensorflow import keras

from keras.layers import GRU, Dropout, SimpleRNN, LSTM, Dense
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Date: The date on which the particular market data was recorded or reported.

Open: The price at which a particular security (stock, currency, etc.) opened for trading at the beginning of the trading day.

High: The highest price at which the security traded during the trading day.

Low: The lowest price at which the security traded during the trading day.

Close: The price at which the security closed trading at the end of the trading day.

Adj Close (Adjusted Close): The adjusted closing price reflects the security's value at the end of the trading day, adjusted for corporate actions (such as stock splits, dividends, etc.) that occurred after the market's close.

Volume: The total number of shares or contracts traded during a given period. It represents the level of activity or liquidity in the market for that security.

In [3]:
google_stock_train_data = pd.read_csv("/kaggle/input/google-stock-2010-2023/Google_Stock_Train (2010-2022).csv", parse_dates=['Date'])

In [4]:
google_stock_train_data.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,15.689439,15.753504,15.621622,15.684434,15.684434,78169752
1,2010-01-05,15.695195,15.711712,15.554054,15.615365,15.615365,120067812
2,2010-01-06,15.662162,15.662162,15.174174,15.221722,15.221722,158988852
3,2010-01-07,15.250250,15.265265,14.831081,14.867367,14.867367,256315428
4,2010-01-08,14.814815,15.096346,14.742492,15.065566,15.065566,188783028
5,2010-01-11,15.126627,15.126627,14.865866,15.042793,15.042793,288227484
6,2010-01-12,14.956206,14.968969,14.714715,14.776777,14.776777,193937868
7,2010-01-13,14.426677,14.724224,14.361862,14.691942,14.691942,259604136
8,2010-01-14,14.612112,14.869870,14.584835,14.761011,14.761011,169434396
9,2010-01-15,14.848348,14.853854,14.465465,14.514515,14.514515,217162620


In [5]:
google_stock_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3272 entries, 0 to 3271
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       3272 non-null   datetime64[ns]
 1   Open       3272 non-null   float64       
 2   High       3272 non-null   float64       
 3   Low        3272 non-null   float64       
 4   Close      3272 non-null   float64       
 5   Adj Close  3272 non-null   float64       
 6   Volume     3272 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 179.1 KB


In [6]:
# Lets see the line chart for google stock prices over time
fig1 = px.line(google_stock_train_data, x='Date', y='Close', title='Google Stock Price Over Time')
fig1.show()

/opt/conda/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [7]:
# Lets take a look at daily trading volume
fig2 = px.scatter(google_stock_train_data, x='Date', y='Volume', title='Daily Trading Volume')
fig2.show()

/opt/conda/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [8]:
# Lets get the five number summary for Google stock prices for each year
google_stock_train_data['Year'] = google_stock_train_data['Date'].dt.year
summary_by_year = google_stock_train_data.groupby('Year')['Close'].describe()
summary_by_year

,count,mean,std,min,25%,50%,75%,max
Year,,,,,,,,
2010,252.0,13.403981,1.262781,10.912663,12.265453,13.371371,14.549112,15.684935
2011,252.0,14.238593,1.072171,11.883884,13.279279,14.450575,15.245057,16.163664
2012,250.0,16.086498,1.324606,13.990240,15.140328,15.835711,17.041041,19.220470
2013,252.0,22.128197,2.470652,17.589338,20.273836,21.928179,22.676051,28.045795
2014,252.0,28.406118,1.262285,24.908001,27.383375,28.511249,29.468100,30.534784
2015,252.0,30.999411,4.359576,24.853001,27.332625,28.694001,34.082375,39.698002
2016,252.0,38.160661,1.870102,34.056999,36.524375,38.087499,39.993249,41.786999
2017,251.0,46.988653,3.768367,40.388500,42.837502,47.377499,49.627001,54.254501
2018,251.0,56.102179,3.479520,49.233501,53.408501,55.602501,58.830999,64.275002


In [9]:
# box plot for Google stock prices for each year
fig3 = px.box(google_stock_train_data, x='Year', y='Close', title='Google Stock Prices - Yearly Box Plot')
fig3.show()

In [10]:
fig4 = go.Figure(data=[go.Candlestick(x=np.array(google_stock_train_data['Date']),
                                     open=google_stock_train_data['Open'],
                                     high=google_stock_train_data['High'],
                                     low=google_stock_train_data['Low'],
                                     close=google_stock_train_data['Close'])])

fig4.update_layout(title='Google Stock Prices - Candlestick Chart',
                   xaxis_title='Date',
                   yaxis_title='Stock Price')

fig4.show()


In [11]:
# Histogram of daily returns
google_stock_train_data['Daily_Return'] = google_stock_train_data['Close'].pct_change()
fig5 = px.histogram(google_stock_train_data, x='Daily_Return', nbins=30, title='Distribution of Daily Returns')
fig5.show()

In [12]:
# Heatmap of correlation matrix
correlation_matrix = google_stock_train_data.corr()
fig6 = px.imshow(correlation_matrix, x=correlation_matrix.index, y=correlation_matrix.columns, title='Correlation Matrix Heatmap')
fig6.show()

In [13]:
# Moving average of closing prices
google_stock_train_data['MA_50'] = google_stock_train_data['Close'].rolling(window=50).mean()
fig7 = px.line(google_stock_train_data, x='Date', y=['Close', 'MA_50'], title='Google Stock Price with 50-Day Moving Average')
fig7.show()

/opt/conda/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [14]:
# Scatter plot of closing prices vs. trading volume
fig8 = px.scatter(google_stock_train_data, x='Close', y='Volume', title='Closing Prices vs. Trading Volume')
fig8.show()

In [15]:
# Line chart for daily stock price change
google_stock_train_data['Daily_Change'] = google_stock_train_data['Close'].diff()
fig9 = px.line(google_stock_train_data, x='Date', y='Daily_Change', title='Daily Stock Price Change')
fig9.show()

/opt/conda/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [16]:
# Bar chart of trading volume per month
google_stock_train_data['Month'] = google_stock_train_data['Date'].dt.month
monthly_volume = google_stock_train_data.groupby('Month')['Volume'].sum().reset_index()
fig10 = px.bar(monthly_volume, x='Month', y='Volume', title='Total Trading Volume per Month')
fig10.show()

In [17]:
# Area chart for daily trading volume
fig11 = px.area(google_stock_train_data, x='Date', y='Volume', title='Daily Trading Volume (Area Chart)')
fig11.show()

/opt/conda/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [18]:
# Histogram of daily stock price changes
fig12 = px.histogram(google_stock_train_data, x='Daily_Change', title='Distribution of Daily Stock Price Changes')
fig12.show()

In [19]:
# Line chart for daily stock price percent change
google_stock_train_data['Daily_Pct_Change'] = google_stock_train_data['Close'].pct_change() * 100
fig13 = px.line(google_stock_train_data, x='Date', y='Daily_Pct_Change', title='Daily Stock Price Percent Change')
fig13.show()

/opt/conda/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [20]:
# Box plot of daily stock price percent change by month
fig14 = px.box(google_stock_train_data, x='Month', y='Daily_Pct_Change', title='Daily Stock Price Percent Change by Month')
fig14.show()